In [24]:
import requests
import geopandas as gpd
import pandas as pd
import numpy as np
import difflib
from shapely.geometry import Point, LineString
import re

import warnings
warnings.filterwarnings("ignore")

class HrHydroLink:
    def __init__(self, feature_id, init_lat, init_lon, init_crs=4269, stream_name=None, buffer_m=1000):
        '''
        Description: Initiates High Resolution variables
        
        Input: x,y coordinate in NAD83 (CRS 4269) or WGS84 (CRS 4326) are recommended
        stream_name works best without abbreviations
        buffer is in meters
        '''
        self.id = feature_id
        self.init_lat = init_lat
        self.init_lon = init_lon
        self.init_crs = init_crs
        self.stream_name = stream_name
        self.buffer_m = buffer_m
        self.message = ''
        self.reach_query = None
        self.reach_df = None
        self.best_reach = {'init_id':feature_id, 'init_stream':stream_name}
        self.no_data = {
            'GNIS_NAME': None,
            'LENGTHKM': None,
            'PERMANENT_IDENTIFIER': None,
            'REACHCODE': None,
            'geometry': None,
            'snap_meas': None,
            'snap_xy': None,
            'hl_snap_meters': None,
            'to_node_meters': None,
            'closest': None,
            'name_check': None,
            'mult_reach_ct': None}
        
    def crs_to_4269(self):
        '''
        Description:
        '''
        if self.init_crs == 4269:
            pass
        else:
            try:
                init_point = Point(self.init_lon, self.init_lat)  
                pt_gdf = gpd.GeoSeries(init_point)
                epsg = f'epsg:{str(self.init_crs)}'
                crs={'init':epsg}
                pt_gdf.crs = crs
                pt_gdf.to_crs({'init':'epsg:4269'})
                self.init_lon = pt_gdf[0].x
                self.init_lat = pt_gdf[0].y
                self.init_crs = 4269
                
            except:
                print ('Issues with provided coordinate system. Consider using a common crs like 4269 (NAD83) or 4326 (WGS84).')
        
    def build_reach_query(self, service='HEM'):
        '''
        Description:use init attributes to build query of nearby reachcodes
        
        Parameters:
        service = define service to use, options "HEM", "TNM_HR", "TNM_HRPlus"
        
        Note:
        HEM Service  includes m values for all reaches (needed for uncertainty measures)
        TNM_HR m values appear to not be included for lower 48, splits flowlines into in-network and non-network and thus may require multiple service calls
        TNM_HRPlus is snap-shot and may lag latest HR edits, includes m values for all reaches, splits flowlines into in-network and non-network and thus may require multiple service calls
        '''
        
        if self.init_lat and (float(self.init_lat) > 24 and float(self.init_lat)<50) and self.init_lon and (float(self.init_lon) < -66 and float(self.init_lon)> -125):
            q = f"geometryType=esriGeometryPoint&inSR={self.init_crs}&geometry={self.init_lon},{self.init_lat}&distance={self.buffer_m}&units=esriSRUnit_Meter&outSR=4269&f=GEOJSON&outFields=GNIS_NAME,LENGTHKM,PERMANENT_IDENTIFIER,REACHCODE&returnM=True"
            if service == 'HEM':
                base_url = 'https://edits.nationalmap.gov/arcgis/rest/services/HEM/NHDHigh/MapServer/1/query?'
                self.reach_query = f"{base_url}{q}"
                self.reach_geojson = requests.get(self.reach_query).json() #bringing data directly into geopandas losses measures
            if service == 'TNM_HR':
                print ('currently the TNM_HR service does not return all needed information, see function note')
                base_url = 'https://hydro.nationalmap.gov/arcgis/rest/services/nhd/MapServer/5/query?'
                self.reach_query = f"{base_url}{q}"
            if service == 'TNM_HRPlus':
                base_url = 'https://hydro.nationalmap.gov/arcgis/rest/services/NHDPlus_HR/MapServer/2/query?'
                self.reach_query = f"{base_url}{q}"
            
    def find_closest_reaches(self, n=10):
        '''
        Description: Retrieve all reaches meeting params. Find reach that is most likely correct for snapping
        '''
        if self.init_lat and (float(self.init_lat) > 18.921 and float(self.init_lat) < 71.407) and self.init_lon and (float(self.init_lon) < -66.968 and float(self.init_lon)> -178.218):
            try:
                gdf = gpd.read_file(self.reach_query) #ideally use self.reach_geojson to create gdf (this was just easier for testing but does not include )
                #if gdf.shape[0]>0:
                if len(gdf.index)>0:
                    init_point = Point(self.init_lon, self.init_lat)    
                    gdf['snap_meas']=gdf.geometry.project(init_point) #for df calculate measure down reach where init point would snap to that reach
                    id_list = list(set(gdf['PERMANENT_IDENTIFIER'].tolist())) #create list of identifiers
                    reach_data = []

                    #Wanted to run interpolate on entire gdf but geopandas is looking for a value and wont run on df               
                    for pid in id_list:
                        reach_gdf = gdf.loc[gdf['PERMANENT_IDENTIFIER']==pid]
                        reach_gdf['snap_xy']=reach_gdf.geometry.interpolate(reach_gdf.iloc[0]['snap_meas']) #for reach find x,y on line closest to input x,y
                        snap_xy = reach_gdf.iloc[0]['snap_xy']

                        # build line representing snap path and measure it
                        snap_point = Point(snap_xy.x, snap_xy.y)
                        hl_snap_meters = build_meas_line(snap_point, init_point, crs={'init':'epsg:4269'})

                        # build lines to terminal nodes of reach and measure them
                        for line in self.reach_geojson['features']:
                            if line['properties']['PERMANENT_IDENTIFIER']==pid:
                                m_range = [x[3] for x in line['geometry']['coordinates']]
                                for node in line['geometry']['coordinates']:
                                    max_m = max(m_range)
                                    min_m = min(m_range)
                                    if node[3]==max_m:
                                        max_coord = Point(node[0], node[1])
                                        to_max_coord_meters = build_meas_line(max_coord, init_point, crs={'init':'epsg:4269'})
                                    elif node[3]==min_m:
                                        min_coord = Point(node[0], node[1]) 
                                        to_min_coord_meters = build_meas_line(min_coord, init_point, crs={'init':'epsg:4269'})
                                to_node_meters = min(to_min_coord_meters, to_max_coord_meters)

                        #build dictionary of results
                        d = reach_gdf.iloc[0].to_dict()
                        d.update({"hl_snap_meters":hl_snap_meters,"to_node_meters":to_node_meters})
                        reach_data.append(d)

                    #keep n number of closest reaches, order them ascendingly and number closest=1 to furthest=n 
                    df = (pd.DataFrame(reach_data)).nsmallest(n,'hl_snap_meters',keep='all')
                    df = df.sort_values(by=['hl_snap_meters'])
                    df = df.reset_index(drop=True)
                    df['closest'] = df.index +1

                    self.reach_df = df
                else:
                    self.message = 'no reaches retrieved'
            except:
                self.message = 'find best reach failed'
        else:
            self.message = 'initial coordinates outside of United States'
    
    def stream_match(self):
        '''
        Description: determine if user supplied stream name matches gnis stream name in nhd
        Note: It would be ideal to use np.where or alike to assign name_check values to all of the rows 
        at one time but the fuzzy match was not working so current work around using itertuples.  Also
        should be able to delete name_check_text.
        '''
        if self.reach_df is not None and hasattr(self, 'reach_df') and hasattr(self, 'stream_name'):
            df = self.reach_df
            if 'GNIS_NAME' in df and self.stream_name is not None:
                #If GNIS is not null and names match assign a 1
                df['name_check'] = np.where((df.GNIS_NAME.notnull()) & (df.GNIS_NAME.str.lower()==self.stream_name.lower()), 1.0, 0)
                for row in df.itertuples():
                    #pid = (row.PERMANENT_IDENTIFIER)
                    if row.name_check == 1:
                        df.at[row.Index, 'name_check_txt'] = 'exact match of stream names'
                        self.best_reach.update({'stream_clean_ref':self.stream_name.lower()})
                    #elif row.GNIS_NAME is None or row.GNIS_NAME == '' :
                    #    df['name_check_txt'] = 'no gnis name'
                    #    df['name_check'] = 0
                    elif row.GNIS_NAME and self.stream_name:
                        gnis= (row.GNIS_NAME).lower()
                        #stream_lc = (item.stream_name).lower()
                        stream_lc = clean_stream_name(self.stream_name)
                        self.best_reach.update({'stream_clean_ref':stream_lc})
                        if 'tributary' not in stream_lc and 'branch' not in stream_lc:
                            match_ratio = difflib.SequenceMatcher(lambda x: x == " ", gnis, stream_lc).ratio()
                            df.at[row.Index, 'name_check'] = match_ratio
                            # If match ratio is greater than 0.75, update df field 'name_check_txt'
                            if match_ratio >= 0.75:
                                df.at[row.Index, 'name_check_txt'] = 'most likely match of stream names based on fuzzy match'
                            # If match ratio is greater than 0.6 and less than 0.6, update df field 'name_check_txt'
                            elif 0.75 > match_ratio >= 0.6:
                                df.at[row.Index, 'name_check_txt'] = 'likely match of stream names based on fuzzy match'                
                        else:
                            df.at[row.Index, 'name_check_txt'] = 'no stream matching occured due to name containing reference to tributary' 
                    else:
                        df.at[row.Index, 'name_check_txt'] = 'stream names are unlikely match'
            else:
                df['name_check_txt'] = 'missing one or both stream names'
                df['name_check'] = 0
    
    def select_best_reach(self):
        '''
        Description: Of returned reaches finds most likely reach based on available information
        *First checks to see if any reaches have an exact match of names
            *If number of reaches with exact name match equals 1 that is the reach to recommend
            *If number of reaches with exact name match > 1 take the one that is closest to the point
                *If more than one reach has exact name match and are equally close to the point grab the first but note that multiple reaches so that we can recommend taking a closer look
        *If no reaches with exact name match then check for fuzzy matches over 0.75 cutoff
        *If number of reaches with fuzzy name match equals 1 that is the reach to recommend
        *If number of reaches with fuzzy name match > 1 take the one that is closest to the point
            *If more than one reach has fuzzy name match and are equally close to the point grab the first one but note that multiple reaches so that we can recommend taking a closer look
        *If fuzzy match < 0.75 just take closest reach.
        '''
        
        if self.reach_df is not None:
            df = self.reach_df
            name_check_1 = df.loc[df['name_check']==1]

            if len(name_check_1.index)==1:
                vals = name_check_1.iloc[0].to_dict()
                vals.update({'mult_reach_ct':len(name_check_1)})
                self.best_reach.update(vals)
            elif len(name_check_1.index)>1:
                closest = name_check_1.nsmallest(1,'hl_snap_meters', keep='all')
                #take first indexed reach in list of closest reaches
                #if multiple reaches have same hl_snap_meters and matching stream names the count will be recorded in "mult_reach_ct" field
                vals = closest.iloc[0].to_dict()
                vals.update({'mult_reach_ct':len(closest)})
                self.best_reach.update(vals)
            elif len(name_check_1.index)==0:
                name_check_lt1 = df.loc[df['name_check']>=0.75]
                if len(name_check_lt1.index) == 1:
                    vals = name_check_lt1.iloc[0].to_dict()
                    vals.update({'mult_reach_ct':len(name_check_lt1)})
                    self.best_reach.update(vals)
                elif len(name_check_lt1.index)>1:
                    closest = name_check_lt1.nsmallest(1,'hl_snap_meters', keep='all')
                    vals = closest.iloc[0].to_dict()
                    vals.update({'mult_reach_ct':len(closest)})
                    self.best_reach.update(vals)
                elif len(name_check_lt1.index)==0:
                    closest = df.nsmallest(1,'hl_snap_meters', keep='all')
                    vals = closest.iloc[0].to_dict()
                    vals.update({'mult_reach_ct':len(closest)})
                    self.best_reach.update(vals)
            else:
                print ('darn it, the method #best reach# did not work')
        
    def get_hem_measure(self):
        '''
        Description:
        '''
        if 'REACHCODE' in self.best_reach.keys():
            hem_get_hr_xy = 'https://edits.nationalmap.gov/arcgis/rest/services/HEM/NHDHigh/MapServer/exts/Vwe_HEM_Soe/HEMPointEvents'
            reach = self.best_reach['REACHCODE']
            x = self.best_reach['snap_xy'].x
            y = self.best_reach['snap_xy'].y
            xy = '{"x":' + str(self.init_lon) + ',"y":' + str(self.init_lat) + ', "spatialReference": {"wkid":4269}}'

            payload = {
                      "point": xy ,
                      "reachcode": reach,
                      "searchToleranceMeters": 5,
                      "outWKID": 4269,
                      "f": "json"}

            hr_xy = requests.post(hem_get_hr_xy,params=payload,verify=False).json()
            if hr_xy['resultStatus'] == 'success' and hr_xy['features']:
                self.hl_reach_meas = hr_xy['features'][0]['attributes']
                meas = self.hr_xy['features'][0]['attributes']['MEASURE']
                self.best_reach.update({"meas": meas})
            else:
                self.message='get_hem_measure failed'
                self.best_reach.update({"meas": None})
        else:
            self.best_reach.update(self.no_data)  
            
        
            
# def get_measure(reachcode, x, y, crs=4269):
#     '''
#     Description:
#     Note: This can be used to validate HEM SOE extension.  
#     '''
#     pass
        
    
def build_meas_line(point1, point2, crs={'init':'epsg:4269'}):
    '''
    Description: where point1 and 2 are shapely points
    '''
    line_geom = LineString([point1, point2]) 
    line_geoseries = gpd.GeoSeries(line_geom)           
    line_geoseries.crs = crs
    line_geoseries=line_geoseries.to_crs({'init':'epsg:5070'})
    line_length_meters = line_geoseries.length[0]
    return line_length_meters

def clean_stream_name(name):
    '''
    Description: replace common abbreviations, this needs improvement but be careful not to replace 
    strings we dont want to this code currently assumes GNIS_NAME never contains abbreviations... 
    something to verify. If you have a better way to do this let me know!!!!
    '''
    stream = name
    stream_lower = stream.lower() + ' '
    stream_lower = re.sub("[\(\[].*?[\)\]]", "", stream_lower)
    stream_lower = stream_lower.replace(' st. ', ' stream')
    stream_lower = stream_lower.replace(' st ', ' stream')
    stream_lower = stream_lower.replace(' rv. ', ' river')
    stream_lower = stream_lower.replace(' rv ', ' river')
    stream_lower = stream_lower.replace('unt ', 'unnamed tributary ')
    stream_lower = stream_lower.replace(' trib. ', ' tributary')
    stream_lower = stream_lower.replace(' trib) ', ' tributary')
    stream_lower = stream_lower.replace(' trib ', ' tributary')
    stream_lower = stream_lower.replace(' ck ', ' creek')
    stream_lower = stream_lower.replace(' ck. ', ' creek')
    stream_lower = stream_lower.replace(' br ', ' branch')
    stream_lower = stream_lower.replace(' br. ', ' branch')
    stream_name = stream_lower.strip()
    return stream_name         


In [25]:
import pandas as pd
import geopandas as gpd
import requests


def import_file(input_file, latitude_field, longitude_field, identifier_field, stream_name_field):
    '''
    Description: Imports CSV or SHP files into a Pandas dataframe. Makes sure that all fields 
    are in the file and converts the file into a dataframe with a standard set of field names.
    
    Input: CSV or Shapefile
    
    Output: Pandas Dataframe
    '''
    #Check to see if the file is a CSV file
    if input_file.endswith('.csv'):
        print ('\n' + 'reading csv file' +'\n')
        try:
            df = pd.read_csv(input_file)
        except KeyError:
            print ('file did not properly import, verify file name and rerun')
            #It would be nice here to reask for inputFileName and then restart at try statement
    
    #If input file is not a CSV check to see if it is a shapefile
    elif input_file.endswith('.shp'):
        print('\n' + 'reading shapefile' + '\n')
        try:
            df = gp.GeoDataFrame.from_file(input_file)
        except KeyError:
            print ('file did not properly import, verify file name and rerun')
            #It would be nice here to reask for inputFileName and then restart at try statement
    
    #If input file is not a CSV or shapefile tell the user that the file type is not excepted
    else:
        print('File type not currently accepted. Please try .csv or .shp')
        
    if latitude_field in df and longitude_field in df and stream_name_field in df and identifier_field in df:
        df = df[[identifier_field, latitude_field, longitude_field, stream_name_field]].copy()
        df = df.rename(columns={identifier_field: 'id', latitude_field: 'lat', longitude_field: 'lon', stream_name_field: 'stream'})
        return df
    else: 
        print ('verify field names and rerun')
        
        


In [26]:
#Import packages needed for code to run
import warnings

warnings.filterwarnings("ignore")


#Set variables needed to run the code

#User inputs variables needed for code to run
#input_file = input("Enter file name, including extension (only accepts .csv and .shp): ")
#latitude_field = input("Enter field name for latitude, note this is case sensitive: ")
#longitude_field = input("Enter field name for longitude, note this is case sensitive: ")
#stream_name_field = input("Enter field name for stream name, note this is case sensitive: ")
#identifier_field = input("Enter field name for identifier, note this is case sensitive: ")

#Alternative to user input, information can be hard coded here
input_file = 'Hydrolink_test_subset50.csv'
latitude_field = 'pt_Y'
longitude_field = 'pt_X'
stream_name_field = 'WATER_NAME'
identifier_field = 'SAMPLE_ID'

In [27]:
#Set variable list to store output data as it is processed
out_data = []

df = import_file(input_file, latitude_field, longitude_field, identifier_field,stream_name_field)   
crs= 4269

for row in df.itertuples():
    #Define variables based on row,field values
    lat = float(row.lat)
    lon = float(row.lon)
    input_id = row.id
    stream = str(row.stream)

    print ('\n'+'working on : ' + str(input_id) ) 

    item = HrHydroLink(input_id, lat, lon, stream_name=stream)
    item.crs_to_4269()
    item.build_reach_query(service='HEM')
    item.find_closest_reaches()
    item.stream_match()
    item.select_best_reach()

    #record data in outData, this will be used to create dataframe
    out_data.append(item.best_reach)



reading csv file


working on : EMAP_223

working on : PAFBC_30156

working on : NRSA_14911

working on : MontgomeryCty_590

working on : NYDEC_411177 0626 01

working on : SHEN_1992-06-15_F_3F143

working on : SHEN_1992-06-30_F_3F023

working on : PAFBC_61380

working on : MDDNRMBSS_PRMO-114-R-2002

working on : MontgomeryCty_1138

working on : NYDEC_493031 0819 01

working on : SRBC_259

working on : PAFBC_63924

working on : SRBC_530

working on : PAFBC_36338

working on : SRBC_245

working on : MDDNRMBSS_WCHE-106-R-2017

working on : NYDEC_711661 1115 02

working on : BaltimoreCty_142

working on : PAFBC_58982

working on : SHEN_1982-10-22_F_2F055

working on : EMAP_567

working on : WVDEP_94242

working on : PAFBC_64011

working on : PAFBC_42168

working on : MontgomeryCty_448

working on : MDDNRMBSS_OXON-101-R-2015

working on : PAFBC_61679

working on : PAFBC_55055

working on : NYDEC_811018 0523 01

working on : SRBC_202

working on : MDDNRMBSS_NANJ-331-S-2002

working on : PA

In [29]:
df = pd.DataFrame(out_data)

In [8]:
#Test one point

init_id = 1
init_lat = 39.59777
init_lon = -79.0566
init_crs = 4326
init_stream = 'SAVAGE R'

item = HrHydroLink(init_id, init_lat, init_lon, init_crs, init_stream)
item.crs_to_4269()
item.build_reach_query(service='HEM')
item.find_closest_reaches()
item.stream_match()
item.select_best_reach()

item.best_reach  

{'init_id': 1,
 'init_stream': 'SAVAGE R',
 'stream_clean_ref': 'savage r',
 'GNIS_NAME': 'Savage River',
 'LENGTHKM': 2.125,
 'PERMANENT_IDENTIFIER': '45646043',
 'REACHCODE': '02070002000185',
 'geometry': <shapely.geometry.linestring.LineString at 0x1e7a4338ac8>,
 'snap_meas': 0.001100132261661136,
 'snap_xy': <shapely.geometry.point.Point at 0x1e7a4338688>,
 'hl_snap_meters': 15.367702072924763,
 'to_node_meters': 98.04634527457966,
 'closest': 1,
 'name_check': 0.8,
 'name_check_txt': 'most likely match of stream names based on fuzzy match',
 'mult_reach_ct': 1}